In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
# 设置随机种子
Random.seed!(123)

TaskLocalRNG()

### Predict-then-optimize 

In [2]:
Random.seed!(123)
# 设置参数
N = 3   # 样本数量（或产品组数）
K = 10   # 每个样本的选择项数量
tau = 10 # 给定常数
N_u = 2

# 预设数据（根据你的实际问题替换这些值）
u = rand(N_u)        # u 向量 ∈ ℝ^N
A = rand(N,N_u)      # a_n ∈ ℝ^N
# B = rand(N,N)        # b_n ∈ ℝ^N
p_dag = rand(N, K)   # p_{nk}^† 数据矩阵

3×10 Matrix{Float64}:
 0.0802658  0.654101   0.271636  0.88463   …  0.862761  0.387374  0.875555
 0.0490284  0.256433   0.688575  0.810734     0.158312  0.771916  0.264568
 0.915866   0.0862789  0.789013  0.257922     0.5403    0.377383  0.901877

In [3]:
B = rand(N, N)  # 随机生成矩阵
for i in 1:N
    for j in 1:N
        if i == j
            B[i, j] = -abs(B[i, j]) * 10  # 对角线元素为负
        else
            B[i, j] = abs(B[i, j])  # 非对角线元素为正
        end
    end
end

# 确保对角线元素大于所在行和列的其他元素之和
for i in 1:N
    row_sum = sum(B[i, :]) - B[i, i]  # 当前行其他元素之和
    col_sum = sum(B[:, i]) - B[i, i]  # 当前列其他元素之和
    B[i, i] = -max(row_sum, col_sum) - 1  # 确保对角线元素更小
end

println("Generated B matrix:")
println(round.(B, digits=2))

Generated B matrix:
[-2.66 0.51 0.41; 0.88 -2.49 0.54; 0.78 0.98 -2.76]


In [ ]:
model = Model(Mosek.Optimizer)
# model = Model(COPT.ConeOptimizer)
# 定义变量
@variable(model, rho_0 >= 0)                       # ρ₀ ≥ 0
@variable(model, rho[1:N] >= 0)                    # ρ_n ≥ 0
@variable(model, v_sigma[1:N_u])                   # ς = ρ₀ * u
@variable(model, v_phi[1:N])                       # φ_n
@variable(model, Y[1:N, 1:K])                      # y_{nk}
@variable(model, Z[1:N, 1:K])                      # z_{nk}
@variable(model, X[1:N, 1:K], Bin)                 # x_{nk} ∈ {0,1}

# 1. 总和约束
@constraint(model, rho_0 + sum(rho) == 1)

# 2. ς = ρ₀ * u
@constraint(model, v_sigma .== rho_0 * u)

# 3. φ_n = ∑_k z_{nk} * p_{nk}^†
@constraint(model, v_phi .== sum(Z .* p_dag, dims=2))

# 4. 辅助变量上下界
for n in 1:N
    for k in 1:K
        # y_{nk} bounds
        @constraint(model, 0 <= Y[n, k])
        @constraint(model, Y[n, k] <= X[n, k])
        @constraint(model, Y[n, k] >= rho[n] - (1 - X[n, k]))
        @constraint(model, Y[n, k] <= rho[n])

        # z_{nk} bounds
        @constraint(model, 0 <= Z[n, k])
        @constraint(model, Z[n, k] <= X[n, k])
        @constraint(model, Z[n, k] >= rho_0 - (1 - X[n, k]))
        @constraint(model, Z[n, k] <= rho_0)
    end
end


@constraint(model, 1 .== sum(X, dims=2))

# X_given = zeros(N, K)  # 给定的 X 矩阵
# X_given[1, :] = [1, 0, 0]
# X_given[2, :] = [0, 1, 0]   
# X_given[3, :] = [0, 0, 1]
# @constraint(model, X .== X_given)

for n in 1:N
    @constraint(model, [rho[n], rho_0, A[n,:]' * v_sigma + B[n,:]' * v_phi] in MOI.ExponentialCone())
end

@objective(model, Min,sum(Y .* p_dag) - tau * sum(rho))

optimize!(model)

Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 276             
  Affine conic cons.     : 3 (9 rows)
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 99              
  Matrix variables       : 0               
  Integer variables      : 30              

Optimizer started.
Mixed integer optimizer started.
Threads used: 8
Presolve started.
Presolve terminated. Time = 0.00, probing time =  0.00
Clique table size: 0
BRANCHES RELAXS   ACT_NDS  DEPTH    BEST_INT_OBJ         BEST_RELAX_OBJ       REL_GAP(%)  TIME  

Objective of best integer solution : Not available.
Best objective bound               : Not available.
Initial feasible solution objective: Undefined
Construct solution objective       : Not employed
User objective cut value           : Not employed
Number of cuts generated    

In [8]:
# 输出状态与目标值
println("求解状态：", termination_status(model))
println("最优目标值：", objective_value(model))

# 输出部分变量（可选）
println("\n估计的变量：")
println("rho_0 = ", value(rho_0))
println("rho = ", value.(rho))
println("X = \n", round.(Int, value.(X)))

求解状态：INFEASIBLE
最优目标值：0.0

估计的变量：
rho_0 = 0.0
rho = [0.0, 0.0, 0.0]
X = 
[0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0]


In [ ]:
argmax_indices = argmax(value.(X), dims=2)  
for n in 1:N
    println("样本 $n 的最大值索引：", argmax_indices[n])
end